In [ ]:
import datetime as dt
import time
import logging as log
from multiprocessing import Pool, TimeoutError

%run BreakSectionManager.ipynb
%run Util.ipynb
%run Constant.ipynb
%run Metric.ipynb
%run Operation.ipynb
%run Account.ipynb
%run Policy_Loader.ipynb
%run Evaluation.ipynb


class RocketRegression:
    
    def __init__(self, policy_name):
        self.policy_name = policy_name
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        self.context = self.policy_loader.get_context()
        
        self.level = self.context['level']
        self.short = self.context['short']
        self.mid = self.context['mid']
        
    def start_regression_on_one_stock(self,symbol,regression_end_date):
        start = time.time()
        sectionManager = BreakSectionManager(symbol,self.context)
        section_df = sectionManager.get_all_break_sections()
        section_df = section_df[section_df['d_s_datetime']<to_datetime(regression_end_date)]
        
        metric = Metric(symbol,self.context)
        
        stock_start_date = metric.get_stock_start_date()
        stock_end_date = metric.get_stock_end_date()
        log.info('stock_start_date ' + stock_start_date+' stock_end_date:'+stock_end_date)
        
        account = Account(stock_start_date,stock_end_date,self.context,self.policy_name)
        operation = Operation(account,symbol,self.context,self.policy_name)
        
        for key,row in section_df.iterrows():
            start_date = row['d_s_date']
            end_date = row['d_e_date']
            
            start_datetime = to_datetime(start_date)
            end_datetime = to_datetime(end_date)

            buy_price = metric.get_cur_price(start_date)
            
            if(self.debug==True):
                print('\nsection '+start_date+'===>'+end_date)
            while(start_datetime<=end_datetime):
                cur_date = start_datetime.strftime(YMD_format)
                
                if(metric.is_today_open(cur_date)):
                    account.daily_audit(cur_date)

                    if(account.can_open_new_stock()==True):
                        if(operation.is_buy_point(cur_date)==True):
                            operation.open_opsition(cur_date)

                    if(account.has_shares()==True):
                        if(operation.is_sell_point(cur_date)==True):
                            operation.sell_stock(cur_date)

                start_datetime += dt.timedelta(days = 1)
                    
                        
        operation_df = operation.get_operations()
        
        evaluation = Evaluation(account,self.context)
        evaluation_df = evaluation.get_evaluation_report(symbol,stock_start_date,stock_end_date,
                                                        self.policy_name,operation_df)

        end = time.time()
        print(symbol+' regression cost time ' + str(round(end-start,1))+' seconds')
        return evaluation_df
    
    
    def start_regression(self,scale,regression_end_date,process_num):
        print('context:' + str(self.context))
        
        evaluation_df = pd.DataFrame(columns=evaluation_columns)
        symbols = get_symbols(scale)
        total = len(symbols) 
        
        pool = Pool(processes=process_num)
        tasks = []
        for symbol in symbols:
            task = pool.apply_async(self.start_regression_on_one_stock, (symbol,regression_end_date))
            tasks.append(task)
            
        for task in tasks:
            one_evaluation = task.get()
            evaluation_df = pd.concat([one_evaluation,evaluation_df], ignore_index=True, sort=False)
        
        pool.close()
        pool.join()
        
        evaluation_df['loss_times'] =evaluation_df['loss_times'].astype('int')
        evaluation_df['deal_count'] =evaluation_df['deal_count'].astype('int')
        return evaluation_df
    

In [71]:
regression = RocketRegression('rocket_break_baseline.json')

regression_end='2019-05-30'
scale=3
process_num=4
evaluation_df = regression.start_regression(scale,regression_end,process_num)
evaluation_df

context:{'short': 40, 'mid': 240, 'scale_start': 0, 'scale_end': 3, 'level': 'Day'}

year_returns
[-27.8, -21.52]
SZ#002506 regression cost time 7.2 seconds

year_returns
[19.45, 58.62, -13.79]
SZ#002343 regression cost time 11.4 seconds

year_returns
[47.25, 25.94]
SZ#002504 regression cost time 13.3 seconds

year_returns
[4.47, -12.91, 12.03]
SZ#002344 regression cost time 14.6 seconds

year_returns
[35.8, 15.74, -21.99]
SZ#002505 regression cost time 16.9 seconds

year_returns
[-20.68, -30.89, -26.07, 29.2, -22.2]
SZ#002181 regression cost time 18.0 seconds

year_returns
[-9.8, -13.89, 14.58, 6.9, -8.23, -12.56]
SZ#002182 regression cost time 18.5 seconds

year_returns
[-58.5, 4.15, 19.38, 59.13]
SZ#002183 regression cost time 19.0 seconds

year_returns
[-8.44, 22.72, 39.66, -21.29]
SZ#002345 regression cost time 20.7 seconds

year_returns
[3.1, -11.68, -12.0, -16.13, -23.03, 6.83, 33.12]
SZ#000803 regression cost time 26.3 seconds

year_returns
[-13.76, -23.8, 25.91, -38.23, 17.61,

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:165: RuntimeWarning: invalid value encountered in double_scalars


SZ#000004 regression cost time 38.2 seconds
SZ#000806 regression cost time 38.6 seconds

year_returns
[32.09, 44.86, -18.88, -11.02, -14.35, -26.71, -23.47, -1.6, 13.83, -18.83]

year_returns
[15.7, -19.02, -17.67, -11.38, 37.35, 21.01]
SZ#002507 regression cost time 26.9 seconds
SZ#000573 regression cost time 40.4 seconds

year_returns
[-38.05, 21.37, 51.04, 38.16, -5.41, -26.34, -34.9, -9.32, 36.51, -16.86]

year_returns
[-18.12, 31.32, 68.62, -6.05, -14.44, 3.42, 22.56, 10.11, 24.29, -9.34]
SZ#002021 regression cost time 41.0 seconds
SZ#000001 regression cost time 41.3 seconds

year_returns
[46.47, 57.7, -16.14, -16.29, 25.36, 55.08, -27.14]
SZ#000005 regression cost time 29.4 seconds

year_returns
[-10.56, 1.96, 64.12, 75.85, 25.19, -13.07, -4.95, 23.58, -30.91, 15.79]
SZ#000807 regression cost time 45.5 seconds

year_returns
[-35.17, -19.3, -4.01, 51.44, -1.96, 11.65, -42.47, 3.38, -29.87]
SZ#002023 regression cost time 47.2 seconds

year_returns
[-21.1, 27.61, -54.92, -11.3, 11.4

,symbol,start_date,end_date,policy_name,short,mid,max_drawdown,avg_year_return,loss_times,year_regular_return,mean_year_return,std_year_return,mean_month_return,std_month_return,profit_month_rate,win_rate,mar,sharp,deal_count
0,SZ#000822,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,56.23,4.65,4,23.31,-0.321000,27.524012,1.642222,14.364033,51.11,40.00,0.08,-0.120658,10
1,SZ#000596,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,190.00,4315.97,2,15147.41,27.610000,44.632038,17.947321,64.628241,57.14,66.67,22.72,0.551398,6
2,SZ#000017,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,50.88,47.51,0,277.11,-0.300000,25.858935,2.811071,22.412777,42.86,100.00,0.93,-0.127615,4
3,SZ#002517,2010-12-07,2019-02-18,rocket_break_baseline.json,40,240,49.99,17.50,1,119.64,5.200000,29.483000,6.582105,34.920967,42.11,33.33,0.35,0.074619,3
4,SZ#002356,2010-02-09,2019-02-18,rocket_break_baseline.json,40,240,55.21,18.10,1,83.20,8.526000,27.510566,2.621200,16.106324,60.00,33.33,0.33,0.200868,3
5,SZ#002194,2007-12-07,2019-02-18,rocket_break_baseline.json,40,240,55.53,-6.54,3,-27.07,-10.061667,15.424943,-1.403030,12.771435,39.39,16.67,-0.12,-0.846789,6
6,SZ#002033,2004-08-25,2019-02-18,rocket_break_baseline.json,40,240,58.76,12.74,1,44.56,4.362500,30.265678,1.898600,13.916222,54.00,71.43,0.22,0.045018,7
7,SZ#000821,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,63.10,51.55,3,212.09,15.005714,31.348872,5.451081,20.082751,67.57,50.00,0.82,0.382971,6
8,SZ#000595,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,71.05,-1.48,3,-8.41,-6.696000,29.045932,0.129250,15.748858,45.00,33.33,-0.02,-0.333816,9
9,SZ#000016,2000-02-14,2019-02-18,rocket_break_baseline.json,40,240,60.99,20.88,2,79.66,6.182222,30.413793,3.148654,17.546219,53.85,62.50,0.34,0.104631,8


In [72]:
evaluation_df.describe()

,max_drawdown,avg_year_return,loss_times,year_regular_return,mean_year_return,std_year_return,mean_month_return,std_month_return,profit_month_rate,win_rate,mar,sharp,deal_count
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,65.836700,104.258300,2.080000,386.116700,2.875698,31.436639,3.146453,22.632750,49.727900,49.695000,1.123500,-0.055886,6.490000
std,45.725437,474.169524,1.368329,1682.160294,13.965953,28.480587,5.250323,23.440543,10.089462,21.345773,4.017316,0.781368,2.576094
min,31.750000,-17.800000,0.000000,-108.300000,-78.905000,4.440631,-5.536032,7.295313,25.000000,0.000000,-0.270000,-6.228845,1.000000
25%,49.580000,0.690000,1.000000,3.130000,-2.843214,23.253442,0.821553,14.736287,44.140000,33.330000,0.010000,-0.212567,5.000000
50%,59.620000,11.595000,2.000000,48.625000,3.352500,28.450264,2.190179,17.284208,50.000000,50.000000,0.210000,0.008137,7.000000
75%,66.587500,34.532500,3.000000,115.605000,9.764167,34.999395,4.366223,21.776507,55.972500,63.542500,0.675000,0.230766,8.250000
max,399.960000,4315.970000,6.000000,15147.410000,42.638889,291.126825,42.025484,225.012353,76.470000,100.000000,30.850000,2.229493,11.000000
